# 事前準備

## ライブラリ

In [2]:
!pip install --upgrade xlrd==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 103 kB 5.1 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [3]:
import urllib.parse
import urllib.request
import json
import pandas as pd
import numpy as np
import xlrd

## estat-API

APIキーの設定

In [86]:
ESTAT_APPID = '724e5b90772a3e9289f41a253e4e7e32438f4fff'

estat-APIからレスポンスを取得する関数

In [87]:
def get_estat_response(params):
  p = params.copy()

  # appId
  p['appId'] = ESTAT_APPID
  
  # url生成
  url = 'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

レスポンスのJSONをDataFrameに整形する関数

In [88]:
def conv_estat_response_to_dataframe(response):
  # CLASS_INF
  CLASS_OBJ = response['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

  # TABLE_INF
  TABLE_INF = response['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']

  # VALUE
  VALUE = response['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']

  # VALUEをDataFrameに変換
  df_res = pd.json_normalize(VALUE)

  # CLASS_OBJのDataFrameを結合
  for d in CLASS_OBJ :
    # DataFrameに変換 '@code','@name'だけ抽出
    df_class = pd.json_normalize(d['CLASS']) 
    df_class = df_class.copy()[['@code','@name']]

    # @codeをキー名に変更 @nameをキー名＋'_name'に変更
    key_name = '@{}'.format(d['@id'])
    df_class = df_class.rename(columns={'@code':key_name,'@name': key_name+'_name'})

    # DataFrameを結合
    df_res = pd.merge(df_res, df_class, on=key_name, how='outer')

  # 統計情報を追加
  df_res['statsDataId'] = TABLE_INF['@id']
  df_res['statsDataName'] = TABLE_INF['STAT_NAME']['$']
  
  return df_res

# 港湾統計（最新年度）

[国土交通省ホームページ](https://www.mlit.go.jp/k-toukei/kowannenpodb.html)にEXCELデータが公開されている。

今回は令和2年度

## 入港船舶

統計年度'time'とEXCELのURL'url'を引数として、DataFrameを作成する関数

In [69]:
def get_ship(time,url):

  # ブックの指定
  input_file_name = url
  input_book = pd.ExcelFile(input_file_name)
  
  # 統計結果は2番目のシート、上から2行は無視
  input_sheet_name = input_book.sheet_names
  df = input_book.parse(input_sheet_name[1],skiprows = 2)
  
  if time == '2020年':
    # 必要な列だけ抽出
    columns = ['都道府県','港湾','種別','合計隻数','合計総トン数']
    df = df[columns]
    
  # 都道府県と港湾で集計
  df = df.groupby(['都道府県','港湾']).agg({'合計隻数': 'sum','合計総トン数': 'sum'}).reset_index()

  # 列名を整形
  df = df.rename(columns={'合計隻数' : f'入港船舶_隻数【{time}】','合計総トン数' : f'入港船舶_総トン数【{time}】'})

  return df

統計年度とEXCELを指定して実行

In [70]:
time = '2020年'
url  = 'https://www.mlit.go.jp/k-toukei/content/001448031.xlsx'

df_ship_2020 = get_ship(time,url)
df_ship_2020

,都道府県,港湾,入港船舶_隻数【2020年】,入港船舶_総トン数【2020年】
0,三重県,吉津,2438,603188
1,三重県,四日市,15986,59791880
2,三重県,尾鷲,3623,191480
3,三重県,津松阪,5654,1715923
4,三重県,鵜殿,793,212705
...,...,...,...,...
161,鹿児島県,川内,2668,2033871
162,鹿児島県,志布志,3124,18436450
163,鹿児島県,米之津,112,107145
164,鹿児島県,西之表,4302,3321652


## 取扱貨物量

統計年度'time'とEXCELのURL'url'を引数として、DataFrameを作成する関数

In [46]:
def get_cargo(time,url):

  # ブックの指定
  input_file_name = url
  input_book = pd.ExcelFile(input_file_name)
  
  # 統計結果は2番目のシート、上から2行は無視
  input_sheet_name = input_book.sheet_names
  df = input_book.parse(input_sheet_name[1],skiprows = 2)

  # 都道府県と港湾で集計
  df = df.groupby(['都道府県','港湾']).agg({'合計': 'sum','輸出': 'sum','輸入': 'sum','移出': 'sum','移入': 'sum'}).reset_index()
  
  # 都道府県の合計値は不要
  df = df[df['港湾'] != '合計']

  # 列名を整形
  df = df.rename(columns={'合計' : f'取扱貨物量【{time}】', '輸出' : f'輸出量【{time}】', '輸入' : f'輸入量【{time}】', '移出' : f'移出量【{time}】', '移入' : f'移入量【{time}】'})

  return df

統計年度とEXCELを指定して実行

In [59]:
time = '2020年'
url  = 'https://www.mlit.go.jp/k-toukei/content/001448034.xlsx'

df_cargo_2020 = get_cargo(time,url)
df_cargo_2020

,都道府県,港湾,取扱貨物量【2020年】,輸出量【2020年】,輸入量【2020年】,移出量【2020年】,移入量【2020年】
1,三重県,吉津,1257638,0.0,0.0,1257216.0,422.0
2,三重県,四日市,56269697,3749166.0,32740679.0,14804177.0,4975675.0
3,三重県,尾鷲,183262,0.0,0.0,1476.0,181786.0
4,三重県,津松阪,1723891,1000.0,186329.0,164143.0,1372419.0
5,三重県,鵜殿,416957,0.0,5500.0,13801.0,397656.0
...,...,...,...,...,...,...,...
200,鹿児島県,川内,1145670,186710.0,625980.0,120879.0,212101.0
201,鹿児島県,志布志,9144176,406261.0,3040524.0,3173562.0,2523829.0
202,鹿児島県,米之津,184428,0.0,105960.0,0.0,78468.0
203,鹿児島県,西之表,1300219,0.0,0.0,552959.0,747260.0


## コンテナ個数

統計年度'time'とEXCELのURL'url'を引数として、DataFrameを作成する関数

In [62]:
def get_container(time,url):

  # ブックの指定
  input_file_name = url
  input_book = pd.ExcelFile(input_file_name)
  
  # 統計結果は2番目のシート、上から2行は無視
  input_sheet_name = input_book.sheet_names
  df = input_book.parse(input_sheet_name[1],skiprows = 2)

  if time == '2020年':
    # 必要な列だけ抽出
    df = df[['都道府県','港湾','計','コンテナ個数','空コンテナ個数']]

    # 1行目は不要
    df = df.drop(df.index[[0]])

  # 都道府県と港湾で集計
  df = df.groupby(['都道府県','港湾']).agg({'計': 'sum','コンテナ個数': 'sum','空コンテナ個数': 'sum'}).reset_index()
  
  # 列名を整形
  df = df.rename(columns={'計' : f'コンテナ個数_合計【{time}】', 'コンテナ個数' : f'コンテナ個数【{time}】', '空コンテナ個数' : f'空コンテナ個数【{time}】'})

  return df

In [63]:
time = '2020年'
url  = 'https://www.mlit.go.jp/k-toukei/content/001448042.xlsx'

df_container_2020 = get_container(time,url)
df_container_2020

,都道府県,港湾,コンテナ個数_合計【2020年】,コンテナ個数【2020年】,空コンテナ個数【2020年】
0,三重県,四日市,210958.0,162538.0,48420.0
1,京都府,舞鶴,19995.0,11934.0,8061.0
2,佐賀県,伊万里,67198.0,40402.0,26796.0
3,兵庫県,姫路,12795.0,8219.0,4576.0
4,兵庫県,東播磨,4665.0,2271.0,2394.0
...,...,...,...,...,...
82,鹿児島県,宮之浦(屋久島町),9345.0,7274.0,2071.0
83,鹿児島県,川内,20540.0,13741.0,6799.0
84,鹿児島県,志布志,92074.0,58879.0,33197.0
85,鹿児島県,西之表,7515.0,7515.0,0.0


## まとめ

入港船舶、取扱貨物量、コンテナ個数を結合する

In [72]:
df_2020 = pd.merge(df_ship_2020, df_cargo_2020, on=['都道府県','港湾'], how='outer')
df_2020 = pd.merge(df_2020, df_container_2020, on=['都道府県','港湾'], how='outer')

df_2020

,都道府県,港湾,入港船舶_隻数【2020年】,入港船舶_総トン数【2020年】,取扱貨物量【2020年】,輸出量【2020年】,輸入量【2020年】,移出量【2020年】,移入量【2020年】,コンテナ個数_合計【2020年】,コンテナ個数【2020年】,空コンテナ個数【2020年】
0,三重県,吉津,2438,603188,1257638,0.0,0.0,1257216.0,422.0,NaN,NaN,NaN
1,三重県,四日市,15986,59791880,56269697,3749166.0,32740679.0,14804177.0,4975675.0,210958.0,162538.0,48420.0
2,三重県,尾鷲,3623,191480,183262,0.0,0.0,1476.0,181786.0,NaN,NaN,NaN
3,三重県,津松阪,5654,1715923,1723891,1000.0,186329.0,164143.0,1372419.0,NaN,NaN,NaN
4,三重県,鵜殿,793,212705,416957,0.0,5500.0,13801.0,397656.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
161,鹿児島県,川内,2668,2033871,1145670,186710.0,625980.0,120879.0,212101.0,20540.0,13741.0,6799.0
162,鹿児島県,志布志,3124,18436450,9144176,406261.0,3040524.0,3173562.0,2523829.0,92074.0,58879.0,33197.0
163,鹿児島県,米之津,112,107145,184428,0.0,105960.0,0.0,78468.0,NaN,NaN,NaN
164,鹿児島県,西之表,4302,3321652,1300219,0.0,0.0,552959.0,747260.0,7515.0,7515.0,0.0


csvファイルをダウンロード

In [ ]:
from google.colab import files

df_2020.to_csv('港湾統計【2020年】.csv', encoding="shift_jis")
files.download('港湾統計【2020年】.csv')

# 製造品出荷額（最新年度）

[工業統計調査](https://www.e-stat.go.jp/dbview?sid=0003448126)の2020年確報データを利用する

In [203]:
params = {
    'statsDataId' : '0003448126',
    'cdCat01' : '32000120',
    'cdCat02' : '00',
}

res = get_estat_response(params)
print(res)

{'GET_STATS_DATA': {'RESULT': {'STATUS': 0, 'ERROR_MSG': '正常に終了しました。', 'DATE': '2022-09-27T14:53:00.148+09:00'}, 'PARAMETER': {'LANG': 'J', 'STATS_DATA_ID': '0003448126', 'NARROWING_COND': {'CODE_CAT01_SELECT': 32000120, 'CODE_CAT02_SELECT': '00'}, 'DATA_FORMAT': 'J', 'START_POSITION': 1, 'METAGET_FLG': 'Y'}, 'STATISTICAL_DATA': {'RESULT_INF': {'TOTAL_NUMBER': 1946, 'FROM_NUMBER': 1, 'TO_NUMBER': 1946}, 'TABLE_INF': {'@id': '0003448126', 'STAT_NAME': {'@code': '00550010', '$': '工業統計調査'}, 'GOV_ORG': {'@code': '00550', '$': '経済産業省'}, 'STATISTICS_NAME': '工業統計調査 確報 ２０２０年確報', 'TITLE': {'@no': '3-07', '$': '地域別 ３．市区町村別産業中分類別統計表\u3000従業者４人以上の事業所に関する統計表（事業所数、従業者数、現金給与総額、原材料使用額等、製造品出荷額等及び粗付加価値額）'}, 'CYCLE': '年次', 'SURVEY_DATE': '202001-202012', 'OPEN_DATE': '2022-03-09', 'SMALL_AREA': 0, 'MAIN_CATEGORY': {'@code': '05', '$': '鉱工業'}, 'SUB_CATEGORY': {'@code': '02', '$': '製造業'}, 'OVERALL_TOTAL_NUMBER': 134096, 'UPDATED_DATE': '2022-07-05', 'STATISTICS_NAME_SPEC': {'TABULATION_CATEGORY': '工業統計調査',

## 全市区町村の値

都道府県の値、政令指定都市分割（例えば神戸市中央区）の値も含まれるので、市区町村の値のみを抽出する

In [204]:
# CLASS_OBJ
CLASS_OBJ = res['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

# 地域一覧を抽出
arealist = list(filter(lambda d : d['@id'] == 'cat03', CLASS_OBJ))[0]['CLASS']

# 市区町村コードのリスト作成（@level=2)
areas = [d.get('@code')
                 for d in filter(lambda x: x['@level'] == '2', arealist)]

print(areas)

['01100', '01202', '01203', '01204', '01205', '01206', '01207', '01208', '01209', '01210', '01211', '01212', '01213', '01214', '01215', '01216', '01217', '01218', '01219', '01220', '01221', '01222', '01223', '01224', '01225', '01226', '01227', '01228', '01229', '01230', '01231', '01233', '01234', '01235', '01236', '01303', '01304', '01331', '01332', '01333', '01334', '01337', '01343', '01345', '01346', '01347', '01361', '01362', '01363', '01364', '01367', '01370', '01371', '01392', '01393', '01394', '01395', '01396', '01397', '01398', '01399', '01400', '01401', '01402', '01404', '01405', '01406', '01407', '01408', '01409', '01423', '01424', '01425', '01427', '01428', '01429', '01430', '01431', '01432', '01433', '01434', '01437', '01438', '01452', '01453', '01454', '01455', '01456', '01457', '01458', '01459', '01460', '01461', '01462', '01463', '01464', '01465', '01468', '01469', '01470', '01471', '01472', '01481', '01482', '01483', '01484', '01485', '01486', '01487', '01511', '01512', 

In [138]:
def get_products_city(params):

  # estat-APIからデータ取得
  res = get_estat_response(params)

  # レスポンスJSONをDataDrameに変換
  df = conv_estat_response_to_dataframe(res)
  
  # 市区町村のみ抽出
  df = df[df['@cat03'].isin(areas)]

  # 必要な列だけ抽出
  df =  df[['@cat03_name','$']]

  # 列名の変更
  columns = {'@cat03_name':'市区町村名','$':'製造品出荷額等'}
  df = df.rename(columns=columns)

  # 市区町村名を整形
  df['市区町村名'] = df.apply(lambda x: x['市区町村名'][7:], 1)

  # 欠損値を処理
  df = df.replace('X', '0')

  # 製造品出荷額を数値に変換して降順にソート
  df = df.astype({'製造品出荷額等': 'int'}).sort_values('製造品出荷額等', ascending=False)

  return df


In [146]:
df_products = get_products_city(params)
df_products

,市区町村名,製造品出荷額等
1098,豊田市,1517165565
747,川崎市,408279669
626,市原市,406664255
728,横浜市,392691150
1469,倉敷市,387861070
...,...,...
397,大沼郡金山町,0
128,天塩郡幌延町,0
1388,吉野郡野迫川村,0
1736,東松浦郡玄海町,0


csvファイルをダウンロード

In [162]:
from google.colab import files

df_products.to_csv('製造品出荷額【2020年】.csv', encoding="shift_jis")
files.download('製造品出荷額【2020年】.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 播磨臨海地域の値

播磨臨海地域を集計する場合

In [158]:
def sum_harimarinkai(df):
  # 播磨臨海地域の定義
  harimarinkai = ['姫路市','加古川市','高砂市','明石市','加古郡播磨町','加古郡稲美町']
  
  # 播磨臨海地域を合計した行を作成
  df = df_products[df_products['市区町村名'].isin(harimarinkai)]
  df = df.append(df.sum(numeric_only=True), ignore_index=True)
  df = df.tail(1)
  df.at[6, "市区町村名"] = '播磨臨海地域'

  return df

In [160]:
df_products_harima = sum_harimarinkai(df_products)
df_products_harima

,市区町村名,製造品出荷額等
6,播磨臨海地域,601470433.0


# 取扱貨物量の推移（総トン数）

## 単年度処理

[国土交通省ホームページ](https://www.mlit.go.jp/k-toukei/kowannenpodb.html)のEXCELデータから作業する


URLからEXCELデータを読み込んでDataFrameに格納

In [193]:
def get_cargo(time,url):

  # ブックの指定
  input_file_name = url
  input_book = pd.ExcelFile(input_file_name)
  
  # 統計結果は2番目のシート、上から2行は無視
  input_sheet_name = input_book.sheet_names
  df = input_book.parse(input_sheet_name[1],skiprows = 2)

  # 都道府県と港湾で集計
  df = df.groupby(['都道府県','港湾']).agg({'合計': 'sum','輸出': 'sum','輸入': 'sum','移出': 'sum','移入': 'sum'})
  
  # 列の値を行にstack
  df = pd.DataFrame(df.stack()).reset_index()

  # 列名の整形
  column = f'取扱貨物量【{time}】'
  df['港湾'] = df.apply(lambda x: '{}_{}'.format(x['都道府県'],x['港湾']) if x['港湾'] != '合計' else x['都道府県'], 1)
  df[column] = df.apply(lambda x: x[0], 1)
  df['種類'] = df.apply(lambda x: x['level_2'], 1)

  # 必要な列だけ抽出
  df = df[['都道府県','港湾','種類',column]]
  
  return df

In [194]:
time = '2020年'
url = 'https://www.mlit.go.jp/k-toukei/content/001448034.xlsx'


df_2020 = get_cargo(time,url)
df_2020

,都道府県,港湾,種類,取扱貨物量【2020年】
0,三重県,三重県,合計,59851445.0
1,三重県,三重県,輸出,3750166.0
2,三重県,三重県,輸入,32932508.0
3,三重県,三重県,移出,16240813.0
4,三重県,三重県,移入,6927958.0
...,...,...,...,...
1020,鹿児島県,鹿児島県_鹿児島,合計,27746958.0
1021,鹿児島県,鹿児島県_鹿児島,輸出,1599.0
1022,鹿児島県,鹿児島県_鹿児島,輸入,1232606.0
1023,鹿児島県,鹿児島県_鹿児島,移出,12309449.0


## 年次推移

パラメータをセットして反復処理

In [196]:
params_cargo = [
    { 'time' : '2012年' , 'url' : 'https://www.mlit.go.jp/common/001277668.xls' },
    { 'time' : '2013年' , 'url' : 'https://www.mlit.go.jp/common/001277684.xls' },
    { 'time' : '2014年' , 'url' : 'https://www.mlit.go.jp/common/001277699.xls' },
    { 'time' : '2015年' , 'url' : 'https://www.mlit.go.jp/common/001380149.xlsx' },
    { 'time' : '2016年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448496.xlsx' },
    { 'time' : '2017年' , 'url' : 'https://www.mlit.go.jp/common/001380093.xlsx' },
    { 'time' : '2018年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448540.xlsx' },
    { 'time' : '2019年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448577.xlsx' },
    { 'time' : '2020年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448034.xlsx' },
]

In [199]:
def get_merge_dataframe(params):
  df_res = pd.DataFrame(index=[], columns=['都道府県','港湾','種類'])
  
  for p in params:
    df = get_cargo(p['time'], p['url'])
    df_res = pd.merge(df_res, df, on=['都道府県','港湾','種類'] , how='outer')

  return df_res

In [200]:
df_cargo = get_merge_dataframe(params_cargo)
df_cargo

,都道府県,港湾,種類,取扱貨物量【2012年】,取扱貨物量【2013年】,取扱貨物量【2014年】,取扱貨物量【2015年】,取扱貨物量【2016年】,取扱貨物量【2017年】,取扱貨物量【2018年】,取扱貨物量【2019年】,取扱貨物量【2020年】
0,三重県,三重県_四日市,合計,61045644.0,62809567.0,63543538.0,62963079.0,61407654.0,59011286.0,60562345.0,60865745.0,56269697.0
1,三重県,三重県_四日市,輸出,2656057.0,5077016.0,4693342.0,4767529.0,4233535.0,3805539.0,4222073.0,4633468.0,3749166.0
2,三重県,三重県_四日市,輸入,38865579.0,38924254.0,38478887.0,37479135.0,36302079.0,35365141.0,35981946.0,35591909.0,32740679.0
3,三重県,三重県_四日市,移出,15065552.0,13668124.0,15072960.0,15216806.0,15374439.0,13764465.0,14778341.0,15068792.0,14804177.0
4,三重県,三重県_四日市,移入,4458456.0,5140173.0,5298349.0,5499609.0,5497601.0,6076141.0,5579985.0,5571576.0,4975675.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1045,長崎県,長崎県_肥前大島,合計,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,379915.0
1046,長崎県,長崎県_肥前大島,輸出,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1047,長崎県,長崎県_肥前大島,輸入,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74825.0
1048,長崎県,長崎県_肥前大島,移出,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12239.0


csvファイルをダウンロード

In [201]:
from google.colab import files

df_cargo.to_csv('取扱貨物量の推移【総トン数】.csv', encoding="shift_jis")
files.download('取扱貨物量の推移【総トン数】.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 取扱貨物量（品目別）

[国土交通省ホームページ](https://www.mlit.go.jp/k-toukei/kowannenpodb.html)のEXCELデータから作業する

輸出・輸入・移出・移入とも同じ構成のExcelになっているので、EXCELのインポートからDataFameの整形までは共通関数でOK

In [202]:
def get_excel(url):
  # ブックの指定
  input_file_name = url
  input_book = pd.ExcelFile(input_file_name)
  
  # 統計結果は2番目のシート、上から2行は無視
  input_sheet_name = input_book.sheet_names
  df = input_book.parse(input_sheet_name[1],skiprows = 2)

  return df

In [ ]:
def get_dataframe(time,url):
 
  # EXCELデータをインポート
  df = get_excel(url)
  
  # 都道府県と港湾で集計
  df = df.astype({'トン数': 'int'})
  df = df.groupby(['都道府県','港名','品種名']).agg({'トン数': 'sum'}).reset_index()

  df = df[df['港名'] != '合計']

  df = df.rename(columns={'トン数':time})
  
  return df

繰り返し処理してひとつのDataFrameに整形する関数

In [ ]:
def get_merge_dataframe(params):
  df_res = pd.DataFrame(index=[], columns=['都道府県','港名','品種名'])
  
  for p in params:
    df = get_dataframe(p['time'], p['url'])
    df_res = pd.merge(df_res, df, on=['都道府県','港名','品種名'] , how='outer')

  return df_res

## 輸出

In [ ]:
params_export = [
    { 'time' : '2012年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448464.xlsx' },
    { 'time' : '2013年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448471.xlsx' },
    { 'time' : '2014年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448478.xlsx' },
    { 'time' : '2015年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448485.xlsx' },
    { 'time' : '2016年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448499.xlsx' },
    { 'time' : '2017年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448521.xlsx' },
    { 'time' : '2018年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448545.xlsx' },
    { 'time' : '2019年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448582.xlsx' },
    { 'time' : '2020年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448037.xlsx' },
]

In [ ]:
df_export = get_merge_dataframe(params_export)
df_export

,都道府県,港名,品種名,2012年,2013年,2014年,2015年,2016年,2017年,2018年,2019年,2020年
0,三重県,四日市,その他日用品,1861.0,1376.0,1393.0,2405.0,2822.0,1466.0,2999.0,1384.0,1069.0
1,三重県,四日市,その他機械,3433.0,1340.0,371.0,239.0,107.0,522.0,107.0,NaN,616.0
2,三重県,四日市,その他畜産品,70.0,477.0,0.0,22.0,200.0,133.0,200.0,265.0,90.0
3,三重県,四日市,その他石油製品,5492.0,23263.0,36.0,98144.0,105.0,321.0,713.0,787.0,845.0
4,三重県,四日市,その他繊維工業品,2709.0,4575.0,2576.0,2118.0,2144.0,953.0,1532.0,84.0,476.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3482,鹿児島県,喜入,重油,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63800.0
3483,鹿児島県,川内,その他雑穀,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0
3484,鹿児島県,志布志,完成自動車,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0
3485,鹿児島県,志布志,米,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6000.0


## 輸入

In [ ]:
params_import = [
    { 'time' : '2012年' , 'url' : 'https://www.mlit.go.jp/common/001277672.xls' },
    { 'time' : '2013年' , 'url' : 'https://www.mlit.go.jp/common/001277688.xls' },
    { 'time' : '2014年' , 'url' : 'https://www.mlit.go.jp/common/001277703.xls' },
    { 'time' : '2015年' , 'url' : 'https://www.mlit.go.jp/common/001277717.xls' },
    { 'time' : '2016年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448500.xlsx' },
    { 'time' : '2017年' , 'url' : 'https://www.mlit.go.jp/common/001277753.xls' },
    { 'time' : '2018年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448546.xlsx' },
    { 'time' : '2019年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448583.xlsx' },
    { 'time' : '2020年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448038.xlsx' },
]

In [ ]:
df_import = get_merge_dataframe(params_import)
df_import

,都道府県,港名,品種名,2012年,2013年,2014年,2015年,2016年,2017年,2018年,2019年,2020年
0,三重県,四日市,その他日用品,58191.0,80136.0,94268.0,122110.0,145715.0,189704.0,212454.0,183452.0,210129.0
1,三重県,四日市,その他林産品,1779.0,784.0,575.0,681.0,498.0,364.0,116.0,228.0,362.0
2,三重県,四日市,その他機械,0.0,NaN,0.0,57.0,57.0,63.0,1402.0,1063.0,82.0
3,三重県,四日市,その他石油製品,1103.0,6333.0,90.0,339.0,18.0,25709.0,168843.0,300259.0,420987.0
4,三重県,四日市,その他繊維工業品,3385.0,2389.0,2193.0,10240.0,6572.0,5883.0,7378.0,12979.0,10314.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4477,鳥取県,境,その他の石油,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9518.0
4478,鳥取県,境,その他輸送用車両,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,175.0
4479,鹿児島県,喜入,重油,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,171013.0
4480,鹿児島県,志布志,ＬＮＧ（液化天然ガス）,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.0


## 移出

In [ ]:
params_emigration = [
    { 'time' : '2012年' , 'url' : 'https://www.mlit.go.jp/common/001277673.xls' },
    { 'time' : '2013年' , 'url' : 'https://www.mlit.go.jp/common/001277689.xls' },
    { 'time' : '2014年' , 'url' : 'https://www.mlit.go.jp/common/001277704.xls' },
    { 'time' : '2015年' , 'url' : 'https://www.mlit.go.jp/common/001277718.xls' },
    { 'time' : '2016年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448501.xlsx' },
    { 'time' : '2017年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448522.xlsx' },
    { 'time' : '2018年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448547.xlsx' },
    { 'time' : '2019年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448584.xlsx' },
    { 'time' : '2020年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448039.xlsx' },
]

In [ ]:
df_emigration = get_merge_dataframe(params_emigration)
df_emigration

,都道府県,港名,品種名,2012年,2013年,2014年,2015年,2016年,2017年,2018年,2019年,2020年
0,三重県,四日市,その他石油製品,379629.0,846192.0,1017790.0,1220640.0,675150.0,610549.0,532937.0,543146.0,512377.0
1,三重県,四日市,とうもろこし,6446.0,35674.0,10000.0,5600.0,NaN,NaN,NaN,1770.0,1467.0
2,三重県,四日市,コークス,4500.0,18962.0,16550.0,42806.0,NaN,6900.0,7682.0,NaN,NaN
3,三重県,四日市,セメント,1256924.0,2749690.0,2790124.0,2014088.0,990337.0,900140.0,982981.0,964573.0,967494.0
4,三重県,四日市,分類不能のもの,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4989,鹿児島県,志布志,事務用機器,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
4990,鹿児島県,志布志,陶磁器,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
4991,鹿児島県,西之表,ゴム製品,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4992,鹿児島県,西之表,水,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


## 移入

In [ ]:
params_immigration = [
    { 'time' : '2012年' , 'url' : 'https://www.mlit.go.jp/common/001277674.xls' },
    { 'time' : '2013年' , 'url' : 'https://www.mlit.go.jp/common/001277690.xls' },
    { 'time' : '2014年' , 'url' : 'https://www.mlit.go.jp/common/001277705.xls' },
    { 'time' : '2015年' , 'url' : 'https://www.mlit.go.jp/common/001380154.xlsx' },
    { 'time' : '2016年' , 'url' : 'https://www.mlit.go.jp/common/001380116.xlsx' },
    { 'time' : '2017年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448523.xlsx' },
    { 'time' : '2018年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448548.xlsx' },
    { 'time' : '2019年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448585.xlsx' },
    { 'time' : '2020年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448040.xlsx' },
]

In [ ]:
df_immigration = get_merge_dataframe(params_immigration)
df_immigration

,都道府県,港名,品種名,2012年,2013年,2014年,2015年,2016年,2017年,2018年,2019年,2020年
0,三重県,四日市,その他石油製品,38076.0,33310.0,1200.0,NaN,18.0,2202.0,105074.0,39662.0,36433.0
1,三重県,四日市,その他食料工業品,3811.0,1300.0,4399.0,6114.0,6139.0,5210.0,4054.0,3908.0,3847.0
2,三重県,四日市,とうもろこし,300.0,NaN,NaN,NaN,1500.0,NaN,3000.0,NaN,2400.0
3,三重県,四日市,コークス,39681.0,43442.0,21919.0,9330.0,6750.0,134848.0,4750.0,650.0,2900.0
4,三重県,四日市,セメント,50726.0,82441.0,56856.0,41049.0,34107.0,34522.0,42325.0,21240.0,40911.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5681,鹿児島県,名瀬,金属鉱,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0
5682,鹿児島県,宮之浦(屋久島町),石灰石,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11020.0
5683,鹿児島県,志布志,事務用機器,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
5684,鹿児島県,西之表,鉄道車両,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0


## 搬出（輸出＋移出）

In [ ]:
df_export = df_export.set_index(['都道府県','港名','品種名'])
df_emigration = df_emigration.set_index(['都道府県','港名','品種名'])

df_carryout = df_export + df_emigration
df_carryout.reset_index()
df_carryout

## 搬入（輸入＋移入）

In [ ]:
df_import = df_import.set_index(['都道府県','港名','品種名'])
df_immigration = df_immigration.set_index(['都道府県','港名','品種名'])

df_carryin = df_import + df_immigration
df_carryin

2012年     2013年     2014年     2015年     2016年  \
都道府県 港名  品種名                                                             
三重県  吉津  水産品               NaN       NaN       NaN       NaN       NaN   
     四日市 その他の石油            NaN       NaN       NaN       NaN       NaN   
         その他日用品            NaN       NaN   94336.0  122130.0  146211.0   
         その他林産品            NaN       NaN       NaN       NaN       NaN   
         その他機械             NaN       NaN       NaN       NaN       NaN   
...                        ...       ...       ...       ...       ...   
鹿児島県 鹿児島 非鉄金属           1831.0       NaN       NaN       NaN      43.0   
         飲料                NaN       NaN       NaN       NaN       NaN   
         麦            181322.0  218399.0  157008.0  188645.0  104550.0   
         ＬＮＧ（液化天然ガス）       NaN       NaN       NaN       NaN       NaN   
         ＬＰＧ（液化石油ガス）  130235.0  122799.0  121693.0  118812.0  112120.0   

                         2017年     2018年     2019年     2020年  
都道府県 港名  品種名                                                  
三重県  吉津  水産品               NaN       NaN       NaN       NaN  
     四日市 その他の石油            NaN  368068.0  374482.0  398453.0  
         その他日用品            NaN       NaN       NaN  210245.0  
         その他林産品            NaN       NaN       NaN       NaN  
         その他機械             NaN       NaN       NaN       NaN  
...                        ...       ...       ...       ...  
鹿児島県 鹿児島 非鉄金属             40.0      34.0       NaN      90.0  
         飲料                NaN       NaN       NaN       NaN  
         麦            140773.0  148985.0       NaN  149324.0  
         ＬＮＧ（液化天然ガス）       NaN       NaN       NaN       NaN  
         ＬＰＧ（液化石油ガス）  115763.0  110652.0  104679.0  100967.0  

[6858 rows x 9 columns]

# 入港船舶

In [ ]:
params_ship = [
    { 'time' : '2012年' , 'url' : 'https://www.mlit.go.jp/common/001277666.xls' },
    { 'time' : '2013年' , 'url' : 'https://www.mlit.go.jp/common/001277682.xls' },
    { 'time' : '2014年' , 'url' : 'https://www.mlit.go.jp/common/001277697.xls' },
    { 'time' : '2015年' , 'url' : 'https://www.mlit.go.jp/common/001380148.xlsx' },
    { 'time' : '2016年' , 'url' : 'https://www.mlit.go.jp/common/001380113.xlsx' },
    { 'time' : '2017年' , 'url' : 'https://www.mlit.go.jp/common/001380092.xlsx' },
    { 'time' : '2018年' , 'url' : 'https://www.mlit.go.jp/common/001380059.xlsx' },
    { 'time' : '2019年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448575.xlsx' },
    { 'time' : '2020年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448031.xlsx' },
]